## Problem 1 - Perceptron

### Answer 1.1

Import `numpy` and `matplotlib`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-whitegrid')

%matplotlib inline

Define needed functions related to generating and plotting 2D data points

In [ ]:
# Function to generate $n$ points of $(x1, x2)$, with range: $low \leq x1 \leq x2 \leq high$
def generate_2D_points(low, high, n_points, seed=None):
    if seed != None:
        np.random.seed(seed)
    
    x1_arr = np.random.uniform(low, high, n_points)
    x2_arr = np.random.uniform(low, high, n_points)

    class_1_data, class_2_data = ([], [])

    for x1, x2 in zip(x1_arr, x2_arr):
        if x1 > x2:
            class_1_data.append((x1, x2))
        else:
            class_2_data.append((x1, x2))

    class_1_data, class_2_data = (np.array(class_1_data), np.array(class_2_data))
    
    return class_1_data, class_2_data

# Function to plot 2D points, color coded based on class (two classes)
def plot_2D_points(class_1_data, class_2_data, class_3_data=[]):
    plt.plot(class_1_data[:, 0], class_1_data[:, 1], 'o', color='green', label='Class 1')

    plt.plot(class_2_data[:, 0], class_2_data[:, 1], 'o', color='blue', label='Class 2')
    
    if len(class_3_data) > 0:
        plt.plot(class_3_data[:, 0], class_3_data[:, 1], 'o', color='red', label='Removed Points')
        
    ax = plt.gca()
    ax.set(xlabel = 'x1', ylabel = 'x2')

    plt.title('2D Scatter Plot for points (x1, x2) in Class 1 (Green) and Class 2 (Blue)')
    plt.legend()
    plt.show()

Generate $20$ points in the `unit square of positive quadrant`, i.e. `Quadrant I` with range: $0 \leq x1 \leq x2 \leq 1$. This is for the training data.

In [ ]:
class_1_train_data, class_2_train_data = generate_2D_points(0, 1, 20, 15)

plot_2D_points(class_1_train_data, class_2_train_data)

Generate $1000$ points in the `unit square of positive quadrant`, i.e. `Quadrant I` with range: $0 \leq x1 \leq x2 \leq 1$. This is for the test data.

In [ ]:
class_1_test_data, class_2_test_data = generate_2D_points(0, 1, 1000, 15)

plot_2D_points(class_1_test_data, class_2_test_data)

Build X and y for train and test, check if the shape matches.


In [ ]:
X_train = np.hstack([class_1_train_data.T, class_2_train_data.T])
X_test = np.hstack([class_1_test_data.T, class_2_test_data.T])

assert X_train.shape == (2, 20)
assert X_test.shape == (2, 1000)

y_train = np.concatenate((np.full(len(class_1_train_data), float(1.0)), np.full(len(class_2_train_data), float(-1.0))), axis=None)
y_test = np.concatenate((np.full(len(class_1_test_data), float(1.0)), np.full(len(class_2_test_data), float(-1.0))), axis=None)

assert y_train.shape == (20,)
assert y_test.shape == (1000,)


Shuffle the data first so later in each minibatch, the Perceptron sees data from both labels

In [ ]:
def shuffle(X, y):
    idxes = np.arange(X.shape[1])
    np.random.shuffle(idxes)
    
    return np.array([X[0][idxes], X[1][idxes]]), y[idxes]

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

Define Perceptron related functions

In [ ]:
# Return label given output using sign function
# x should be an NDARRAY, not a scalar
def sign(x):
    return np.array([1.0 if val >= 0 else -1.0 for val in x])

# Predict label using sign activation
def predict(W, X):
    return sign(np.dot(W.T, X))

# Init weights randomly using normal distribution
def init_weights(w_ranges, seed=None):
    if seed != None:
        np.random.seed(seed)
        
    w1 = np.random.uniform(w_ranges[0][0], w_ranges[0][1], size=None)
    w2 = np.random.uniform(w_ranges[1][0], w_ranges[1][1], size=None)
    
    return np.array([w1, w2])

# X should be in the shape of (n_features, n_data)
# y should be in the shape of (n_data,)
# Function to compute loss
def forward_propagate(W, X, y, algo):
    n_data = X.shape[1]
    
    # a equals 0 for Perceptron Criterion and 1 for Hinge-Loss
    if algo == 'Perceptron criterion':
        a = np.zeros(n_data)
    
    elif algo == 'Hinge loss':
        a = np.full(n_data, 1)
        
    else:
        print('Invalid algorithm')
        assert False
    
    outputs = np.dot(W.T, X)
    assert outputs.shape == (n_data,)
    
    # IMPORTANT: Do ELEMENT-WISE multiplication for y against the output
    diffs = a - np.multiply(y, outputs)
    assert diffs.shape == (n_data,)
    
    losses = np.maximum(np.zeros(n_data), diffs)
    assert losses.shape == (n_data,)
    
    # IMPORTANT: Don't forget to divide the losses 
    # by the total number of data so that the updates
    # are not EXTREME
    losses = np.divide(losses, n_data)
    assert losses.shape == (n_data,)

    return outputs, losses

# X should be in the shape of (n_features, n_data)
# y should be in the shape of (n_data,)
# Function to update weights
def back_propagate(W, X, y, outputs, losses, learning_rate):
    y_preds = sign(outputs)
    assert y_preds.shape == (X.shape[1],)
    
    diff = np.subtract(y, y_preds)
    assert diff.shape == (X.shape[1],)
    
    # Update only when loss > 0
    for i, loss in enumerate(losses):
        if loss > 0.0:
            # IMPORTANT: Slice the second axis
            curr_X = X[:, i]
            assert curr_X.shape == (X.shape[0],)
            
            update_values = np.multiply(learning_rate, np.multiply(curr_X, diff[i]))
            assert update_values.shape == (X.shape[0],)

            W = np.add(W, update_values)
            assert W.shape == (X.shape[0],)
    
    return W

Training related functions

In [ ]:
# Train loop, stops when error is zero or when max_iter is reached
def train(X, y, algo, learning_rate, max_iter, w_ranges, seed, debug):
    W = init_weights(w_ranges, seed)
    
    print('Initial Weights:')
    print('w1: {:.2f}, w2: {:.2f}'.format(W[0], W[1]))
    print('-----------------------')

    i = 0
    
    while True:
        outputs, losses = forward_propagate(W, X_train, y_train, algo)
        W = back_propagate(W, X_train, y_train, outputs, losses, learning_rate)
        
        avg_losses = np.mean(losses)
        misc = compute_performance(W, X_train, y_train, 'misc')
        
        if algo == 'Perceptron criterion':
            if misc == 0:
                break
                
        elif algo == 'Hinge loss':
            if avg_losses == 0.0:
                break
                
        else:
            print('Invalid algorithm')
            assert False
        
        i = i + 1
        
        if i < max_iter:
            if debug[0]:
                if i % debug[1] == 0:
                    print('Iteration: {}'.format(i))
                    print('Number of misclassified examples: {}'.format(misc))
                    print('w1: {:.2f}, w2: {:.2f}'.format(W[0], W[1]))
                    print('-----------------------')
        
        else:
            break
    
    print('{} stops at iteration: {}'.format(algo, i))
    print('Number of misclassified examples: {}'.format(misc))
    print('w1: {:.2f}, w2: {:.2f}'.format(W[0], W[1]))
    print('-----------------------')
    
    return W
   
# Call predict to get labels and then compute test accuracy
def compute_performance(W, X, y, ret_type):
    y_preds = predict(W, X)
    
    verdict = [1 if val1 == val2 else 0 for val1, val2 in zip(y_preds, y)]
    
    if ret_type == 'misc':
        return len(y) - np.sum(verdict)
    elif ret_type == 'acc':
        return np.sum(verdict) / len(y)

Train for Perceptron criterion

In [ ]:
algo = 'Perceptron criterion'
learning_rate = 0.1
n_iters = 100 # Stop when loss equals zero or reach n_iters
debug = (False, None)

# Initial value range for (w1, w2), uniform random from -0.1 to 0.1
w_ranges = ((-0.1, 0.1), (-0.1, 0.1)) 
seed = 8

W_pc = train(X_train, y_train, algo, learning_rate, n_iters, w_ranges, seed, debug)
test_acc = compute_performance(W_pc, X_test, y_test, 'acc')

print('Test accuracy using Perceptron criterion is: {}'.format(test_acc))
print('-----------------------')

### Answer 1.2

Train for Hinge losss

In [ ]:
algo = 'Hinge loss'
learning_rate = 0.25
n_iters = 100 # Stop when loss equals zero or reach n_iters
debug = (False, None)

# Initial value range for (w1, w2), uniform random from -0.1 to 0.1
w_ranges = ((-0.1, 0.1), (-0.1, 0.1))
seed = 8

W_hl = train(X_train, y_train, algo, learning_rate, n_iters, w_ranges, seed, debug)
test_acc = compute_performance(W_hl, X_test, y_test, 'acc')

print('Test accuracy using Hinge loss is: {}'.format(test_acc))
print('-----------------------')

### Answer 1.3

Question:

In which case do you obtain better accuracy and why?

Answer:

I obtained better test accuracy using Hinge loss (0.974) compared to Perceptron criterion (0.952). 

We know that Perceptron criterion only updates the weights when there is a discrepancy between the predicted sign, i.e. $\hat{y} = sign(\bar{W}.\bar{X})$, and the true sign, i.e. $y$. Thus, the decision boundary produces by the Perceptron doesn't really maximize the margin between the two classes: $y=-1$ and $y=1$.

Meanwhile, Hinge loss also updates the weights when there is a discrepancy between the predicted output, i.e. $\bar{W}.\bar{X}$, and the actual output, denoted by $1 - y(\bar{W}.\bar{X})$. Thus, it maximizes the margin between the two classes and is more robust to the actual data/test set than Perceptron criterion.

### Answer 1.4

Question:

In which case do you think that the classification of the same 1000 test instances will not change significantly by using a different set of 20 training points? (2)

Answer:

In Hinge loss case, with the similar reasoning as part 1.3. We know that Hinge loss always maximizes the margin between the two classes. 

Since Hinge loss always maximize the margin, the resulting decision boundary will remain similar. Meanwhile, in the Perceptron case, the resulting decision boundary very much depends on the location of the training data for each class. 

Take a look again at the training data.

In [ ]:
plot_2D_points(class_1_train_data[[1, 3, 4]], class_2_train_data, class_1_train_data[[0, 2]])

Suppose that we remove the two points near the true decision boundary $x_1 - x_2 = 0$ from Class 1, denoted by the red dots. In that case, we can infer that the Perceptron might have a decision boundary that is close towards the green region, say something like $2x_1 - 3x_2 = 0$. This decision boundary is not close to the true decision boundary of $x_1 - x_2 = 0$. Thus, Perceptron criterion classification will likely to change significantly given different training data.

## Problem 2 - Weight Initialization, Dead Neurons, Leaky ReLU

### Answer 2.1

Check that we are using `Tensorflow 2.3`, otherwise some functions like `tf.keras.initializers.HeNormal` are not available

In [ ]:
!pip show tensorflow

Codebase from https://github.com/Intoli/intoli-article-materials/tree/master/articles/neural-network-initialization

In [ ]:
import pandas as pd

import seaborn as sns
from matplotlib import rcParamsDefault

import tensorflow.keras
from tensorflow.keras import initializers

from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras import backend as K

In [ ]:
def grid_axes_it(n_plots, n_cols=3, enumerate=False, fig=None):
    """
    Iterate through Axes objects on a grid with n_cols columns and as many
    rows as needed to accommodate n_plots many plots.
    Args:
        n_plots: Number of plots to plot onto figure.
        n_cols: Number of columns to divide the figure into.
        fig: Optional figure reference.
    Yields:
        n_plots many Axes objects on a grid.
    """
    n_rows = n_plots / n_cols + int(n_plots % n_cols > 0)

    if not fig:
        default_figsize = rcParamsDefault['figure.figsize']
        fig = plt.figure(figsize=(
            default_figsize[0] * n_cols,
            default_figsize[1] * n_rows
        ))

    for i in range(1, n_plots + 1):
        ax = plt.subplot(n_rows, n_cols, i)
        yield ax


def create_mlp_model(
    n_hidden_layers,
    dim_layer,
    input_shape,
    n_classes,
    kernel_initializer,
    bias_initializer,
    activation,
):
    """Create Multi-Layer Perceptron with given parameters."""
    model = Sequential()
    model.add(Dense(dim_layer, input_shape=input_shape, kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))
    for i in range(n_hidden_layers):
        model.add(Dense(dim_layer, activation=activation, kernel_initializer=kernel_initializer,
                        bias_initializer=bias_initializer))
    model.add(Dense(n_classes, activation='softmax', kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))
    return model


def compile_model(model):
    model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                  optimizer=tensorflow.keras.optimizers.RMSprop(),
                  metrics=['accuracy'])
    return model


def get_activations(model, x, mode=0.0):
    """Extract activations with given model and input vector x."""
    outputs = [layer.output for layer in model.layers]
    activations = K.function([model.input], outputs)
    output_elts = activations([x, mode])
    return output_elts

Divide the main routine from the original source code into some functions below, for reusability and readability

In [ ]:
def generate_mnist_data(flatten=False, newaxis=False):
    # Load and prepare MNIST dataset.
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    data_dim = 784
    n_train = len(X_train)
    n_test = len(X_test)
    n_classes = len(np.unique(y_test))
    
    if flatten:
        X_train = X_train.reshape(n_train, data_dim).astype('float32')[:n_train]
        X_test = X_test.reshape(n_test, data_dim).astype('float32')[:n_test]
        
        if newaxis:
            X_train = X_train[:, :, np.newaxis]
            X_test = X_test[:, :, np.newaxis]
    else:
        X_train = X_train.astype('float32')[:n_train]
        X_test = X_test.astype('float32')[:n_test]
        
        if newaxis:
            X_train = X_train[:, :, :, newaxis]
            X_test = X_test[:, :, :, np.newaxis]
        
    X_train /= 255.0
    X_test /= 255.0

    y_train = tensorflow.keras.utils.to_categorical(y_train, n_classes)
    y_test = tensorflow.keras.utils.to_categorical(y_test, n_classes)

    return X_train, y_train, X_test, y_test, n_classes, data_dim

def train_model(params, X):
    activation_data = []
    
    if params['kernel_init'] == 'random_normal':
        print('Using Random Normal')
        kernel_init = initializers.RandomNormal(mean=0.0, stddev=params['stddev'], seed=params['seed'])
    elif params['kernel_init'] == 'glorot_normal':
        print('Using Glorot Normal')
        kernel_init = initializers.GlorotNormal(seed=params['seed'])
    elif params['kernel_init'] == 'he_normal':
        print('Using He Normal')
        kernel_init = initializers.HeNormal(seed=params['seed'])
    else:
        print('Invalid initializer')
        assert False

    model = create_mlp_model(
        params['n_hidden_layers'],
        params['dim_layer'],
        (params['data_dim'],),
        params['n_classes'],
        kernel_init,
        params['bias_init'],
        params['activation']
    )
    
    compile_model(model)
    
    output_elts = get_activations(model, X)
    
    n_layers = len(model.layers)
    
    i_output_layer = n_layers - 1

    for i, out in enumerate(output_elts[:-1]):
        if i > 0 and i != i_output_layer:
            for out_i in out.ravel()[::20]:
                activation_data.append([i, stddev, out_i])
                
    return activation_data   

def plot_activations_random_normal(df, sigmas, activation):
    # Plot previously saved activations from the 5 hidden layers
    # using different initialization schemes.
    fig = plt.figure(figsize=(15, 12))

    axes = grid_axes_it(len(sigmas), 1, fig=fig)

    for i, sig in enumerate(sigmas):
        ax = next(axes)
        ddf = df[df['Standard Deviation'] == sig]
        sns.violinplot(x='Hidden Layer', y='Output', data=ddf, ax=ax, scale='count', inner=None)

        ax.set_title('Weights Drawn from $N(\mu = 0, \sigma = {:.2f})$ using {} activation function in hidden layers'.format(sig, activation), fontsize=13)

        ax.set_xlabel("Hidden Layer")

        if i == len(sigmas) // 2: 
            ax.set_ylabel("Neuron Activations")

        if i != len(sigmas) - 1:
            ax.set_xticklabels(())

    plt.tight_layout()
    plt.show()
    
def plot_activations_glorot_and_he_normal(df, init, activation):
    # Plot previously saved activations from the 5 hidden layers
    # using different initialization schemes.
    fig = plt.figure(figsize=(15, 12))

    axes = grid_axes_it(1, 1, fig=fig)
    ax = next(axes)
    
    sns.violinplot(x='Hidden Layer', y='Output', data=df, ax=ax, scale='count', inner=None)

    ax.set_title('Weights Drawn from {} Uniform using {} activation function in hidden layers'.format(init, activation), fontsize=13)

    ax.set_xlabel("Hidden Layer")
    ax.set_ylabel("Neuron Activations")
    ax.set_xticklabels(())

    plt.tight_layout()
    plt.show()


Vanishing Gradients Phenomenon using Random Normal Initialization for Tanh and Sigmoid Activation Function

In [ ]:
# Generate train and test data from MNIST
X_train, y_train, X_test, y_test, n_classes, data_dim = generate_mnist_data(flatten=True, newaxis=False)

# MLP Params in relation to the dataset
params = {
    'n_hidden_layers': 5,
    'dim_layer': 100,
    'data_dim': data_dim,
    'n_classes': n_classes,
    'kernel_init': 'random_normal',
    'bias_init': 'zeros',
    'seed': 10
}

# Run the data through a few MLP models and save the activations from
# each layer into a Pandas DataFrame.
activation_data = []
activations = ['Tanh', 'Sigmoid']
sigmas = [1.0]

for activation in activations:
    print(activation + ' Hidden Layer Neuron Activation Plots')
    print('-------------------')
    
    params['activation'] = activation.lower()
    
    for stddev in sigmas:
        params['stddev'] = stddev
        
        for activation_datum in train_model(params, X_train):
            activation_data.append(activation_datum)

    df = pd.DataFrame(activation_data, columns=['Hidden Layer', 'Standard Deviation', 'Output'])

    plot_activations_random_normal(df, sigmas, activation)

We can see that ...

Fixing Vanishing Gradients Phenomenon using Glorot Uniform Initialization for Tanh and Sigmoid Activation Function

In [ ]:
# Update 'kernel_init', i.e. weight initializer, in MLP Params
params['kernel_init'] = 'glorot_normal' 

# Empty activation data
activation_data = []

# Add ReLU activation function
activations.append('ReLU')

for activation in activations:
    print(activation + ' Hidden Layer Neuron Activation Plots')
    print('-------------------')
    
    params['activation'] = activation.lower()
        
    for activation_datum in train_model(params, X_train):
        activation_data.append(activation_datum)

    df = pd.DataFrame(activation_data, columns=['Hidden Layer', 'Standard Deviation', 'Output'])
    df.drop(['Standard Deviation'], axis=1)

    plot_activations_glorot_and_he_normal(df, 'Glorot Normal', activation)

We can see that...

Finally, we're using He initializers with ReLU activation function in hidden layers

In [ ]:
# Update 'kernel_init', i.e. weight initializer, in MLP Params
params['kernel_init'] = 'he_normal' 

# Empty activation data
activation_data = []

# Only use ReLU
activations = ['ReLU']

for activation in activations:
    print(activation + ' Hidden Layer Neuron Activation Plots')
    print('-------------------')
    
    params['activation'] = activation.lower()
        
    for activation_datum in train_model(params, X_train):
        activation_data.append(activation_datum)

    df = pd.DataFrame(activation_data, columns=['Hidden Layer', 'Standard Deviation', 'Output'])
    df.drop(['Standard Deviation'], axis=1)

    plot_activations_glorot_and_he_normal(df, 'He Normal', activation)

### Answer 2.2

Generate dataset related functions

In [ ]:
import time

from tensorflow.keras.layers import ReLU, LeakyReLU

In [ ]:
def f_x(x):
    return np.multiply(x, np.sin(np.multiply(x, 5.0)))

def generate_dataset(n_data):
    X = np.random.uniform(-1.0 * np.sqrt(7), np.sqrt(7), n_data)
    assert X.shape == (n_data,)

    y = f_x(X)
    assert y.shape == (n_data,)
    
    return X, y

def train_test_split(X, y, train_ratio):
    n_data = len(X)
    assert n_data == len(y)
    
    idx_split = int(train_ratio * n_data)
    
    return X[:idx_split], y[:idx_split], X[idx_split:], y[idx_split:]

Enable parallel training, each model training will now take only 0.1 fraction of the GPU

In [ ]:
# import tensorflow as tf
# from tensorflow.compat.v1.keras.backend import set_session

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.1 # set 0.3 to what you want
# set_session(tf.Session(config=config))

Define network and its training

In [ ]:
def build_relu_model(params):
    # Create the model as sequential
    model = Sequential()

    # First hidden layer
    model.add(Dense(
        params['n_hidden_units'], 
        input_shape=(params['feature_vector_length'],), 
        activation=params['activation'],
        kernel_initializer=params['kernel_init'],
        bias_initializer=params['bias_init']
    ))

    # The rest of the hidden layers
    for i in range(params['n_hidden_layers']-1):
        model.add(Dense(params['n_hidden_units'], activation=params['activation']))

    # Output layer
    model.add(Dense(1, activation='linear'))

    # Compile, use MAE loss, Adam optimizer
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=params['metrics'])
        
    return model

def train_dying_relu(params):
    collapse_count = 0
    
    if params['timed']:
        start = time.time()
    
    for i in range(0, params['n_experiments']):
        X, y = generate_dataset(params['n_data'])
        
        X_train, y_train, X_test, y_test = train_test_split(X, y, 0.8)
        
        # Make sure weights are different every time we do the independent experiment
        model = build_relu_model(params)
        
        # Train using minibatches
        model.fit(X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], verbose=params['verbose'])
        
        # Check if collapsed
        if is_collapsed(model, X_test, params['threshold']):
            collapse_count += 1
        
        if (i+1) % 100 == 0:
            print('Experiment number: {}'.format(i+1))
            
            if params['timed']:
                end = time.time()

                print('Elapsed time: {}'.format(end - start))
                
                start = time.time()
        
    return collapse_count / params['n_experiments'] * 100

# Define functions to check if network collapses
def is_collapsed(network, X_test, threshold):
    y_preds = network.predict(X_test).flatten()
    
    if np.var(y_preds) <= threshold:
        return True
    
    return False

In [ ]:
# Training configs
params = {
    'feature_vector_length': 1,
    'n_hidden_layers': 10,
    'n_hidden_units': 2,
    'batch_size': 64,
    'activation': ReLU(),
    'kernel_init': initializers.HeNormal(),
    'bias_init': initializers.Zeros(),
    'loss': 'mean_absolute_error',
    'optimizer': 'adam',
    'metrics': 'mean_squared_error',
    'epochs': 10,
    'verbose': 0,
    'timed': True,
    'n_data': 3000,
    'n_experiments': 1000,
    'threshold': 1e-4 # To determine if network collapses
}

collapse_percentage = train_dying_relu(params)

print('Collapse percentage for ReLU is: {:.2f}%'.format(collapse_percentage))

### Answer 2.3

Just change the `activation` to `LeakyReLU` with `alpha=0.01`

In [ ]:
# Changes only the activation
params['activation'] = LeakyReLU(alpha=0.01)

leaky_collapse_percentage = train_dying_relu(params)

print('Collapse percentage for Leaky ReLU with $N(\alpha = 0.01$ is: {:.2f}%'.format(leaky_collapse_percentage))

We can see that the collapse percentage is now reduced to ...% from ...%. So, Leaky ReLU does help in preventing dying neurons.

## Problem 3 - Batch Normalization, Dropout, MNIST

### Answer 3.1

Question:

Explain the terms co-adaptation and internal covariance-shift. Use examples if needed. You may need to refer to two papers mentioned below to answer this question.

Answer:

1. Co-Adaptation: In deep learning/neural networks, Co-Adaptation happens when different hidden units have similar behavior so that they might cancel each other out [1]. It is not good because it is computationally inefficient and we desire different hidden units to function differently, i.e. to detect independent features. For example, suppose we have two hidden units, each have four incoming weights from the previous units/nodes, denoted by $[w_{11}, ..., w_{14}]$ and $[w_{21}, ..., w_{24}]$ below: $$w_{1}=[−0.82,1.23,−2.34,0.82]$$ $$w_2=[−0.82,−1.20,2.29,0.82]$$ We can see that $w_{12}$ cancels $w_{22}$ and $w_{13}$ cancels $w_{23}$ since their absolute values are similar but they are of different signs. In this case, we actually only need 2 previous nodes to approximate the function, but we have 4 nodes instead. This is indesirable because it means those four weights have high coupling with each other. Not only inefficient, but in the case of one the weights is rendered invalid (e.g. the server crashes and loses its value), then the other couldn't take over the job since they represent more or less the same feature and the network fails, i.e. producing very different output. Dropout mechanism solves this problem.

2. Internal Covariance-Shift: In deep learning/neural networks, Internal Covariance-Shift is defined as the change in the distribution of network activations due to the change in network parameters/weights during training [2]. We know that the output of the first layer is fed into the second layer and so on. Thus, when the weights are updated, the distribution of inputs to subsequent layers will follow suit. Since hidden layers will try to cope with the new distribution every iteration, then the training will be slower, particularly when the statistical distribution of the input to the networks is drastically different from the previous one. For example, consider MLP that is trained with MNIST data in grayscale images. If we now try to apply this MLP to colored handwritten digits, then it will not perform. Both are handwritten digits but obviously with different statistics (mean and variance). To sum up, if a neural network learns some mapping $X \rightarrow Y$ and the distribution of $X$ changes, then we might need to retrain the network by aligning the distribution of $X$ with the distribution of $Y$. Batch Normalization solves this problem.

Reference:

1. https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf
2. https://arxiv.org/pdf/1502.03167.pdf

### Answer 3.2

Define Le-Net 5

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers import Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
def build_lenet_5(batch_norm_params, dropout_params):
    model = Sequential()

    # Add the first convolution layer
    model.add(Conv2D(
        filters = 20,
        kernel_size = (5, 5),
        padding = "same",
        input_shape = (28, 28, 1)))
    
    if batch_norm_params['prev_to_act'] and batch_norm_params['input_layer']:
        if batch_norm_params['input_norm_type'] == 'batch_norm':
            model.add(BatchNormalization())
        
    if dropout_params['prev_to_act'] and dropout_params['input_layer']:
        model.add(Dropout(dropout_params['input_layer_rate']))

    # Add a ReLU activation function
    model.add(Activation(
        activation = "relu"))
    
    if not batch_norm_params['prev_to_act'] and batch_norm_params['input_layer']:
        if batch_norm_params['input_norm_type'] == 'batch_norm':
            model.add(BatchNormalization())
        
    if not dropout_params['prev_to_act'] and dropout_params['input_layer']:
        model.add(Dropout(dropout_params['input_layer_rate']))

    # Add a pooling layer
    model.add(MaxPooling2D(
        pool_size = (2, 2),
        strides =  (2, 2)))

    # Add the second convolution layer
    model.add(Conv2D(
        filters = 50,
        kernel_size = (5, 5),
        padding = "same"))
    
    if batch_norm_params['prev_to_act'] and batch_norm_params['conv_layers']:
        model.add(BatchNormalization())
        
    if dropout_params['prev_to_act'] and dropout_params['conv_layers']:
        model.add(Dropout(dropout_params['conv_layers_rate']))

    # Add a ReLU activation function
    model.add(Activation(
        activation = "relu"))
    
    if not batch_norm_params['prev_to_act'] and batch_norm_params['conv_layers']:
        model.add(BatchNormalization())
        
    if not dropout_params['prev_to_act'] and dropout_params['conv_layers']:
        model.add(Dropout(dropout_params['conv_layers_rate']))

    # Add a second pooling layer
    model.add(MaxPooling2D(
        pool_size = (2, 2),
        strides = (2, 2)))

    # Flatten the network
    model.add(Flatten())

    # Add a fully-connected hidden layer
    model.add(Dense(500))
    
    if batch_norm_params['prev_to_act'] and batch_norm_params['hidden_layers']:
        model.add(BatchNormalization())
        
    if dropout_params['prev_to_act'] and dropout_params['hidden_layers']:
        model.add(Dropout(dropout_params['hidden_layers_rate']))

    # Add a ReLU activation function
    model.add(Activation(
        activation = "relu"))
    
    if not batch_norm_params['prev_to_act'] and batch_norm['hidden_layers']:
        model.add(BatchNormalization())
        
    if not dropout_params['prev_to_act'] and dropout_params['hidden_layers']:
        model.add(Dropout(dropout_params['hidden_layers_rate']))

    # Add a fully-connected output layer
    model.add(Dense(10))

    # Add a softmax activation function
    model.add(Activation("softmax"))

    # Compile the network
    model.compile(
        loss = "categorical_crossentropy", 
        optimizer = SGD(lr = 0.01),
        metrics = ["accuracy"])
    
    return model

def train_le_net_5(X_train, y_train, params):  
    if batch_norm_params['input_layer']:
        if batch_norm_params['input_norm_type'] == 'standard_norm':
            layer = preprocessing.Normalization()

            layer.adapt(X_train)

            X_train = layer(X_train)
        
    model = build_lenet_5(params['batch_norm_params'], params['dropout_params'])
    
    model.fit(X_train, y_train, batch_size=params['batch_size'], epochs=params['nb_epoch'], verbose=params['verbose'])
    
    return model

def evaluate_le_net_5(model, X_test, y_test, params):
    (loss, accuracy) = model.evaluate(test_data, test_labels, batch_size=params['batch_size'], verbose=params['verbose'])
    
    return accuracy

In [ ]:
X_train, y_train, X_test, y_test, n_classes, data_dim = generate_mnist_data(flatten=False, newaxis=True)

assert X_train.shape == (60000, 28, 28, 1)
assert X_test.shape == (10000, 28, 28, 1)

In [ ]:
batch_norm_params = {
    'input_layer': True, 
    'input_norm_type': 'standard_norm', # Use standard normalization for input
    'conv_layers': True,
    'hidden_layers' : True,
    'prev_to_act': True # Place batch norm layer before activation
}

# Don't apply Dropout now
dropout_params = {
    'input_layer': False,
    'conv_layers': False,
    'hidden_layers' : False,
    'prev_to_act': True
}

params = {
    'batch_size': 128,
    'nb_epoch': 20,
    'verbose': 1,
    'batch_norm_params': batch_norm_params,
    'dropout_params': dropout_params
}

model = train_le_net_5(X_train, y_train, params)
test_acc = evaluate_le_net_5(model, X_test, y_test, params)

print('Test accuracy: {}'.format(test_acc))

### Answer 3.3

In [ ]:
# Now use batch norm for input as well
batch_norm_params['input_norm_type']: 'batch_norm'

# Reassign batch_norm_params
params['batch_norm_params'] = batch_norm_params

model = train_le_net_5(X_train, y_train, params)
test_acc = evaluate_le_net_5(model, X_test, y_test, params)

print('Test accuracy: {}'.format(test_acc))

### Answer 3.4

In [ ]:
# Now invalidate batch_norm_params
batch_norm_params = {
    'input_layer': False, # Use standard norm instead 
    'conv_layers': False,
    'hidden_layers' : False,
    'prev_to_act': True # Place batch norm layer before activation
}

# Apply Dropout now
dropout_params = {
    'input_layer': True,
    'conv_layers': True,
    'hidden_layers' : True,
    'prev_to_act': True,
    'input_layer_rate': 0.2,
    'conv_layers_rate': 0.5,
    'hidden_layers_rate': 0.5
}

# Reassign both batch_norm_params and dropout_params
params['batch_norm_params'] = batch_norm_params
params['dropout_params'] = dropout_params

model = train_le_net_5(X_train, y_train, params)
test_acc = evaluate_le_net_5(model, X_test, y_test, params)

print('Test accuracy: {}'.format(test_acc))

### Answer 3.5

In [ ]:
# Now use batch_norm_params again
batch_norm_params = {
    'input_layer': True, 
    'input_norm_type': 'batch_norm', # Use batch norm to normalize input
    'conv_layers': True,
    'hidden_layers' : True,
    'prev_to_act': True # Place batch norm layer before activation
}

# Reassign batch_norm_params
params['batch_norm_params'] = batch_norm_params

model = train_le_net_5(X_train, y_train, params)
test_acc = evaluate_le_net_5(model, X_test, y_test, params)

print('Test accuracy: {}'.format(test_acc))

## Problem 4 - Learning Rate, Batch Size, FashionMNIST

### Answer 4.1

### Answer 4.2

### Answer 4.3

### Answer 4.4

### Answer 4.5

### Answer 4.6

### Answer 4.7